In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [1]:
from datasets import load_dataset

# Load the dataset
ds = load_dataset("lmsys/lmsys-chat-1m")

# Examine the structure
print(ds)
print(ds["train"].features)

DatasetDict({
    train: Dataset({
        features: ['conversation_id', 'model', 'conversation', 'turn', 'language', 'openai_moderation', 'redacted'],
        num_rows: 1000000
    })
})
{'conversation_id': Value(dtype='string', id=None), 'model': Value(dtype='string', id=None), 'conversation': [{'content': Value(dtype='string', id=None), 'role': Value(dtype='string', id=None)}], 'turn': Value(dtype='int64', id=None), 'language': Value(dtype='string', id=None), 'openai_moderation': [{'categories': {'harassment': Value(dtype='bool', id=None), 'harassment/threatening': Value(dtype='bool', id=None), 'hate': Value(dtype='bool', id=None), 'hate/threatening': Value(dtype='bool', id=None), 'self-harm': Value(dtype='bool', id=None), 'self-harm/instructions': Value(dtype='bool', id=None), 'self-harm/intent': Value(dtype='bool', id=None), 'sexual': Value(dtype='bool', id=None), 'sexual/minors': Value(dtype='bool', id=None), 'violence': Value(dtype='bool', id=None), 'violence/graphic': Value(dty

In [2]:
len(ds["train"]["conversation"][0])

2

In [3]:
sample1 = ds["train"]["conversation"][0]

In [4]:
sample1

[{'content': 'how can identity protection services help protect me against identity theft',
  'role': 'user'},
 {'content': "Identity protection services can help protect you against identity theft in several ways:\n\n1. Monitoring: Many identity protection services monitor your credit reports, public records, and other sources for signs of identity theft. If they detect any suspicious activity, they will alert you so you can take action.\n2. Credit freeze: Some identity protection services can help you freeze your credit, which makes it more difficult for thieves to open new accounts in your name.\n3. Identity theft insurance: Some identity protection services offer insurance that can help you recover financially if you become a victim of identity theft.\n4. Assistance: Many identity protection services offer assistance if you become a victim of identity theft. They can help you file a police report, contact credit bureaus, and other steps to help you restore your identity.\n\nOverall

In [5]:
sample2 = ds["train"]["conversation"][1]
sample2

[{'content': "Beside OFAC's selective sanction that target the listed individiuals and entities, please elaborate on the other types of US's sanctions, for example, comprehensive and sectoral sanctions. Please be detailed as much as possible",
  'role': 'user'},
 {'content': "The United States has a number of different types of sanctions that it can use to achieve its foreign policy goals, including both selective and comprehensive sanctions.\n\nSelective sanctions are targeted at specific individuals or entities that are believed to be engaged in activities that are contrary to US interests. These sanctions can take a variety of forms, including asset freezes, travel bans, and restrictions on financial transactions. The Office of Foreign Assets Control (OFAC) is the US government agency responsible for implementing and enforcing these types of sanctions.\n\nComprehensive sanctions, on the other hand, are more broadly based and aim to restrict entire sectors of a country's economy or t

In [6]:
processed_conv = ""
processed_conversations = []

for turn in sample2:
    if turn["role"] == "user":
        processed_conv += f"User: {turn['content']}\n"
    elif turn["role"] == "assistant":
        processed_conv += f"Assistant: {turn['content']}\n"
processed_conversations.append(processed_conv.strip())

In [7]:
processed_conversations

["User: Beside OFAC's selective sanction that target the listed individiuals and entities, please elaborate on the other types of US's sanctions, for example, comprehensive and sectoral sanctions. Please be detailed as much as possible\nAssistant: The United States has a number of different types of sanctions that it can use to achieve its foreign policy goals, including both selective and comprehensive sanctions.\n\nSelective sanctions are targeted at specific individuals or entities that are believed to be engaged in activities that are contrary to US interests. These sanctions can take a variety of forms, including asset freezes, travel bans, and restrictions on financial transactions. The Office of Foreign Assets Control (OFAC) is the US government agency responsible for implementing and enforcing these types of sanctions.\n\nComprehensive sanctions, on the other hand, are more broadly based and aim to restrict entire sectors of a country's economy or to impose a broad ban on certa

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("state-spaces/mamba-2.8b-hf")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
tokenized = tokenizer(processed_conversations, padding=True, truncation=True, return_tensors="pt")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [10]:
tokenized

{'input_ids': tensor([[ 6989,    27, 33007,  ...,  6717,  5567,    27]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}

In [11]:
de_tokenized = tokenizer.decode(tokenized["input_ids"][0])
de_tokenized

"User: Beside OFAC's selective sanction that target the listed individiuals and entities, please elaborate on the other types of US's sanctions, for example, comprehensive and sectoral sanctions. Please be detailed as much as possible\nAssistant: The United States has a number of different types of sanctions that it can use to achieve its foreign policy goals, including both selective and comprehensive sanctions.\n\nSelective sanctions are targeted at specific individuals or entities that are believed to be engaged in activities that are contrary to US interests. These sanctions can take a variety of forms, including asset freezes, travel bans, and restrictions on financial transactions. The Office of Foreign Assets Control (OFAC) is the US government agency responsible for implementing and enforcing these types of sanctions.\n\nComprehensive sanctions, on the other hand, are more broadly based and aim to restrict entire sectors of a country's economy or to impose a broad ban on certai

In [12]:
max_tokenizer = tokenizer(processed_conversations, padding="max_length", truncation=True, max_length=512, return_tensors="pt")
max_tokenizer

{'input_ids': tensor([[ 6989,    27, 33007,   504,  3481,  1934,   434, 13687, 25162,   326,
          2303,   253,  7117, 14374,    74,   780,    84,   285, 14429,    13,
          4496, 21184,   327,   253,   643,  3510,   273,  1982,   434, 17634,
            13,   323,  1650,    13, 11088,   285,  8776,   267, 17634,    15,
          7764,   320,  7000,   347,  1199,   347,  1896,   187,  6717,  5567,
            27,   380,  1986,  2077,   556,   247,  1180,   273,  1027,  3510,
           273, 17634,   326,   352,   476,   897,   281,  5115,   697,  5639,
          3646,  7342,    13,  1690,  1097, 13687,   285, 11088, 17634,    15,
           187,   187, 10004,   422, 17634,   403, 10522,   387,  2173,  4292,
           390, 14429,   326,   403,  6566,   281,   320,  9583,   275,  4712,
           326,   403, 10214,   281,  1982,  6284,    15,  2053, 17634,   476,
          1379,   247,  5235,   273,  4948,    13,  1690, 15231,  1959, 13505,
            13,  4288, 44464,    13,  

In [13]:
de_max_tokenized = tokenizer.decode(max_tokenizer["input_ids"][0])
de_max_tokenized

"User: Beside OFAC's selective sanction that target the listed individiuals and entities, please elaborate on the other types of US's sanctions, for example, comprehensive and sectoral sanctions. Please be detailed as much as possible\nAssistant: The United States has a number of different types of sanctions that it can use to achieve its foreign policy goals, including both selective and comprehensive sanctions.\n\nSelective sanctions are targeted at specific individuals or entities that are believed to be engaged in activities that are contrary to US interests. These sanctions can take a variety of forms, including asset freezes, travel bans, and restrictions on financial transactions. The Office of Foreign Assets Control (OFAC) is the US government agency responsible for implementing and enforcing these types of sanctions.\n\nComprehensive sanctions, on the other hand, are more broadly based and aim to restrict entire sectors of a country's economy or to impose a broad ban on certai

In [14]:
len(de_max_tokenized)

2653

In [15]:
len(de_tokenized)

8304

In [16]:
tokenizer.eos_token = "<|endoftext|>"
tokenizer.bos_token = "<|startoftext|>"
processed_conv = tokenizer.bos_token + " " 
skip_conversation = False
for turn in sample2:
    if turn['role'] == 'user':
        processed_conv += f"Human: {turn['content']}\n"
    elif turn['role'] == 'assistant':
        if not turn['content'].strip():  # Check if the assistant's response is empty
            skip_conversation = True
            break
        processed_conv += f"Assistant: {turn['content']}\n"

if not skip_conversation:
    processed_conv += " " + tokenizer.eos_token  # End token at the end of each conversation
    processed_conversations.append(processed_conv.strip())

processed_conversations

["User: Beside OFAC's selective sanction that target the listed individiuals and entities, please elaborate on the other types of US's sanctions, for example, comprehensive and sectoral sanctions. Please be detailed as much as possible\nAssistant: The United States has a number of different types of sanctions that it can use to achieve its foreign policy goals, including both selective and comprehensive sanctions.\n\nSelective sanctions are targeted at specific individuals or entities that are believed to be engaged in activities that are contrary to US interests. These sanctions can take a variety of forms, including asset freezes, travel bans, and restrictions on financial transactions. The Office of Foreign Assets Control (OFAC) is the US government agency responsible for implementing and enforcing these types of sanctions.\n\nComprehensive sanctions, on the other hand, are more broadly based and aim to restrict entire sectors of a country's economy or to impose a broad ban on certa

In [24]:
tok = tokenizer(processed_conversations, padding=True, truncation=True, return_tensors="pt")

In [25]:
tok

{'input_ids': tensor([[ 6989,    27, 33007,  ...,  6717,  5567,    27]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}

In [27]:
from datasets import load_dataset

def preprocess_single_conversation(conversation):
    processed_conv = ""
    skip_conversation = False
    for turn in conversation:
        if turn['role'] == 'user':
            processed_conv += f"Human: {turn['content']}\n"
        elif turn['role'] == 'assistant':
            if not turn['content'].strip():  # Check if the assistant's response is empty
                skip_conversation = True
                break
            processed_conv += f"Assistant: {turn['content']}\n"
    
    return None if skip_conversation else processed_conv.strip()

# Load the dataset
sample = sample2

print("Original conversation:")
print(sample)
print("\n" + "="*50 + "\n")

# Process the single conversation
processed = preprocess_single_conversation(sample)

print("Processed conversation:")
if processed is None:
    print("This conversation was skipped due to empty assistant response.")
else:
    print(processed)


Original conversation:
[{'content': "Beside OFAC's selective sanction that target the listed individiuals and entities, please elaborate on the other types of US's sanctions, for example, comprehensive and sectoral sanctions. Please be detailed as much as possible", 'role': 'user'}, {'content': "The United States has a number of different types of sanctions that it can use to achieve its foreign policy goals, including both selective and comprehensive sanctions.\n\nSelective sanctions are targeted at specific individuals or entities that are believed to be engaged in activities that are contrary to US interests. These sanctions can take a variety of forms, including asset freezes, travel bans, and restrictions on financial transactions. The Office of Foreign Assets Control (OFAC) is the US government agency responsible for implementing and enforcing these types of sanctions.\n\nComprehensive sanctions, on the other hand, are more broadly based and aim to restrict entire sectors of a co

TypeError: list indices must be integers or slices, not str

In [18]:
from transformers import AutoTokenizer

def preprocess_function(conversation):
    processed_conv = ""
    
    tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b") 
    tokenizer.eos_token = "<|endoftext|>"
    tokenizer.pad_token = tokenizer.eos_token
    
    for turn in conversation:
        if turn['role'] == 'user':
            processed_conv += f"H: {turn['content'].strip()}\n"
        elif turn['role'] == 'assistant':
            processed_conv += f"A: {turn['content'].strip()}\n"
    
    processed_conv += tokenizer.eos_token  # End token at the end of the conversation
    
    tokenized = tokenizer(processed_conv, padding=True, truncation=True, return_tensors="pt")
    
    return processed_conv, tokenized

def test_preprocessing():
    # Sample conversation
    sample_conversation = [
        {"role": "user", "content": "Hello, how are you?"},
        {"role": "assistant", "content": "I'm doing well, thank you for asking. How can I assist you today?"},
        {"role": "user", "content": "Can you explain what machine learning is?"},
        {"role": "assistant", "content": "Certainly! Machine learning is a subset of artificial intelligence..."}
    ]

    # Apply preprocessing
    processed_conv, tokenized = preprocess_function(sample_conversation)

    # Print results
    print("Processed conversation:")
    print(processed_conv)
    print("\nTokenized output:")
    print(f"Input IDs shape: {tokenized['input_ids'].shape}")
    print(f"Attention mask shape: {tokenized['attention_mask'].shape}")

    # Decode the tokenized output to verify
    decoded = tokenizer.decode(tokenized['input_ids'][0])
    print("\nDecoded tokenized output:")
    print(decoded)

# Run the test
test_preprocessing()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Processed conversation:
H: Hello, how are you?
A: I'm doing well, thank you for asking. How can I assist you today?
H: Can you explain what machine learning is?
A: Certainly! Machine learning is a subset of artificial intelligence...
<|endoftext|>

Tokenized output:
Input IDs shape: torch.Size([1, 55])
Attention mask shape: torch.Size([1, 55])

Decoded tokenized output:
H: Hello, how are you?
A: I'm doing well, thank you for asking. How can I assist you today?
H: Can you explain what machine learning is?
A: Certainly! Machine learning is a subset of artificial intelligence...
<|endoftext|>
